(Rough) Stochastic Volatility Simulation
==

Consider the rough volatility model

$$S_t = \mathcal{E} \left( \int_0^t\sqrt{V_u}\mathrm{d}B_u \right), \quad V_t = \mathcal{E}(W_t^\alpha), \quad B_t = \rho W_t + \bar{\rho}\bar{W}_t.$$

We wish to efficiently evaluate

$$C(t, K) := \mathbb{E}[(S_t - K)^+ | \mathcal{F}_0].$$

We will use antithetic variates, conditional expectations and control variates.

**Antithetic variates**

This simply amounts to utilising the four processes $(\pm W, \pm \bar{W})$, which have equivalent law.

We should find $\tilde{\mathbb{V}}[C] = (1 + \rho) \mathbb{V}[C]$, thus providing variance reduction when antithetic observations are negatively correlated.

**Conditional expectation**

Let $BS(S, v; K)$ be the Black-Scholes call formula for spot $S$ and integrated variance $v$ (*e.g.* $v = \sigma_{BS}^2t$).

Writing $S_t = \mathcal{E} \left(\rho \int_0^t\sqrt{V_u}\mathrm{d}W_u \right)\mathcal{E} \left(\bar{\rho} \int_0^t\sqrt{V_u}\mathrm{d}\bar{W}_u \right):= S^W_t S^\bar{W}_t$, we may then make use of

\begin{align} 
C(t, K) &= \mathbb{E}[  \mathbb{E}[(S_t - K)^+ | \mathcal{F}^W_t \wedge \mathcal{F}^B_0] | \mathcal{F}_0] \\
        &= \mathbb{E} \left[  BS \left(S^W_t, \bar{\rho}^2 \int_0^t V_u \mathrm{d}u; K \right) \Big| \mathcal{F}^W_0 \right].
\end{align}

We should find $\tilde{\mathbb{V}}[C] = \mathbb{V}[C] - \mathbb{E}[\mathbb{V}[BS(\bullet)]]$, thus *always* providing a variance reduction.

**Control variates**

Here we instead evaluate

$$C(t, K) := \mathbb{E}[(S_t - K)^+ + c(X_t - \mathbb{E}[X_t])| \mathcal{F}_0].$$

Selecting $c$ optimally, we should find $\tilde{\mathbb{V}}[C] = (1 - \rho ^2) \mathbb{V}[C]$. $X_t$ is selected as the value of a timer option at time $t$. Known to be a martingale, its future expectation is simply its present (model-independent) price.

In full, for timer budget $Q$, we have

$$C(t, K) = \mathbb{E} \left[  (S_t - K)^+ + c\left\{BS \left(S_t, Q -  \int_0^t V_u \mathrm{d}u; K \right) - BS \left(S_0, Q; K \right) \right\}\Big| \mathcal{F}_0 \right].$$

**Combining everything**

It is possible to combine all of the above. Care must be taken to select the most sensible control variate, which becomes a timer option written (theoretically) on the parallel price proces, $S_t^W$. In full, omitting the strike,

$$C(t) = \mathbb{E} \left[  BS \left(S^W_t, \bar{\rho}^2 \int_0^t V_u \mathrm{d}u \right) + c BS \left(S^W_t, \rho^2 \left( Q -  \int_0^t V_u \mathrm{d}u \right) \right) \Big| \mathcal{F}^W_0 \right] - c BS \left(S^W_0, \rho^2 Q \right).$$

Multiple controls
==

We now aim to enhance the effect of controls and also generalise Bergomi's (a little bit inelegant) approach.

We need to allow our previous control to have *any* budget, $Q$, rather than one which exceeds what is achieved along sample paths. (The effectiveness of this earlier approach could be, undesirably, very dependent on not realising outlier variance paths.)

Select an arbitrary budget, $Q > 0$. The control derived from this budget needs defining along paths which exceed it. For option maturity $t$, define the stopping time $\tau_Q := \inf\left\{s > 0: \int_0^s V_u \mathrm{d}u \geq Q \right\}$.

Now we select $BS \left(S_{t}, Q -  \int_0^{t} V_u \mathrm{d}u; K \right)^{\tau_Q} := BS \left(S_{t \wedge \tau_Q}, Q -  \int_0^{t \wedge \tau_Q} V_u \mathrm{d}u; K \right)$ as control for $(S_t - K)^+$. Then we may write

$$C(t) = \mathbb{E} \left[  (S_t - K)^+ + c BS \left(S_{\tau_Q}, Q -  \int_0^{\tau_Q} V_u \mathrm{d}u \right)  \Big| \mathcal{F}^W_0 \right] - c BS \left(S_0, Q \right).$$

Extending this to multiple controls, we simply have

$$C(t) = \mathbb{E} \left[  (S_t - K)^+ + \textbf{c} BS \left(S_{\tau_Q}, \textbf{Q} -  \int_0^{\tau_Q} V_u \mathrm{d}u \right)  \Big| \mathcal{F}^W_0 \right] - \textbf{c} BS \left(S_0, \textbf{Q} \right).$$

In which $\textbf{c}$ takes on the optimal form from p 142 of Stochastic Simulation.

First, I shall implement this for a single general $Q$ before extending to multiple.